# Converted Data Censored CSV

In [1]:
import pandas as pd

df = pd.read_csv("csv/data_censored.csv")  # Load dataset
df.head()  # Check the first few rows

,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0


In [2]:
def define_estimand(data, estimand):
    if estimand == "PP":
        # Per-Protocol: Filter data where eligible == 1
        return data[data['eligible'] == 1]
    elif estimand == "ITT":
        # Intention-to-Treat: Use all data
        return data
    else:
        raise ValueError("Estimand must be 'PP' or 'ITT'")

# Example usage
trial_pp = define_estimand(df, "PP")
trial_itt = define_estimand(df, "ITT")

In [3]:
def prepare_data(data, id_col, period_col, treatment_col, outcome_col, eligible_col):
    return data[[id_col, period_col, treatment_col, outcome_col, eligible_col]]

# Example usage
trial_pp_prepared = prepare_data(trial_pp, 'id', 'period', 'treatment', 'outcome', 'eligible')
trial_itt_prepared = prepare_data(trial_itt, 'id', 'period', 'treatment', 'outcome', 'eligible')

In [5]:
import statsmodels.api as sm

def calculate_weights(data, numerator_formula, denominator_formula, censor_event_col):
    """
    Calculate inverse probability of censoring weights (IPCW).
    
    Parameters:
        data (pd.DataFrame): The dataset.
        numerator_formula (str): Formula for the numerator model.
        denominator_formula (str): Formula for the denominator model.
        censor_event_col (str): Column name for the censoring indicator.
    
    Returns:
        pd.Series: Weights for each observation.
    """
    # Fit numerator model
    numerator_model = sm.Logit(1 - data[censor_event_col], sm.add_constant(data[numerator_formula])).fit(disp=0)
    numerator_probs = numerator_model.predict(sm.add_constant(data[numerator_formula]))
    
    # Fit denominator model
    denominator_model = sm.Logit(1 - data[censor_event_col], sm.add_constant(data[denominator_formula])).fit(disp=0)
    denominator_probs = denominator_model.predict(sm.add_constant(data[denominator_formula]))
    
    # Calculate weights
    weights = numerator_probs / denominator_probs
    return weights

# Example usage for Per-Protocol
# Define the formulas for numerator and denominator models
numerator_formula_pp = ['age']  # Example: Only age is used in the numerator
denominator_formula_pp = ['age', 'x1', 'x3']  # Example: Age, x1, and x3 are used in the denominator

# Calculate weights for Per-Protocol
trial_pp.loc[:, 'weights'] = calculate_weights(trial_pp, numerator_formula_pp, denominator_formula_pp, 'censored')

# Example usage for Intention-to-Treat
# Define the formulas for numerator and denominator models
numerator_formula_itt = ['x2']  # Example: Only x2 is used in the numerator
denominator_formula_itt = ['x2', 'x1']  # Example: x2 and x1 are used in the denominator

# Calculate weights for Intention-to-Treat
trial_itt.loc[:, 'weights'] = calculate_weights(trial_itt, numerator_formula_itt, denominator_formula_itt, 'censored')

/var/folders/nq/03snv6_j3gb6_kdg7vcn9r200000gn/T/ipykernel_74735/3031175995.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_pp.loc[:, 'weights'] = calculate_weights(trial_pp, numerator_formula_pp, denominator_formula_pp, 'censored')


In [ ]:
from lifelines import CoxPHFitter

def fit_outcome_model(data, outcome_col, treatment_col, covariates, weights_col):
    # Check if the required columns exist in the DataFrame
    required_columns = [outcome_col, treatment_col, weights_col] + covariates
    missing_columns = [col for col in required_columns if col not in data.columns]
    
    if missing_columns:
        raise KeyError(f"The following columns are missing in the DataFrame: {missing_columns}")
    
    # Prepare the data for the CoxPHFitter
    data = data[required_columns]
    
    # Fit the Cox proportional hazards model
    cph = CoxPHFitter()
    cph.fit(data, duration_col=outcome_col, event_col=outcome_col, weights_col=weights_col)
    return cph

# Example usage
covariates = ['x2', 'age']

# Check if the 'outcome' column exists in trial_pp and trial_itt
if 'outcome' not in trial_pp.columns or 'outcome' not in trial_itt.columns:
    raise KeyError("The column 'outcome' is missing in the DataFrame. Please check your data.")

# Fit the outcome models
outcome_model_pp = fit_outcome_model(trial_pp, 'outcome', 'treatment', covariates, 'weights')
outcome_model_itt = fit_outcome_model(trial_itt, 'outcome', 'treatment', covariates, 'weights')

# Print model summaries
outcome_model_pp.print_summary()
outcome_model_itt.print_summary()

In [ ]:
def expand_trials(data, id_col, period_col, treatment_col, outcome_col, weights_col):
    expanded_data = []
    for patient_id, patient_data in data.groupby(id_col):
        for period in range(patient_data[period_col].max() + 1):
            expanded_data.append({
                'id': patient_id,
                'period': period,
                'treatment': patient_data[treatment_col].iloc[0],
                'outcome': patient_data[outcome_col].iloc[0],
                'weights': patient_data[weights_col].iloc[0]
            })
    return pd.DataFrame(expanded_data)

# Example usage
expanded_trial_pp = expand_trials(trial_pp, 'id', 'period', 'treatment', 'outcome', 'weights')
expanded_trial_itt = expand_trials(trial_itt, 'id', 'period', 'treatment', 'outcome', 'weights')

In [ ]:
def fit_msm(data, outcome_col, treatment_col, covariates, weights_col):
    cph = CoxPHFitter()
    data = data[[outcome_col, treatment_col] + covariates + [weights_col]]
    cph.fit(data, duration_col=outcome_col, event_col='outcome', weights_col=weights_col)
    return cph

# Example usage
msm_pp = fit_msm(expanded_trial_pp, 'outcome', 'treatment', covariates, 'weights')
msm_itt = fit_msm(expanded_trial_itt, 'outcome', 'treatment', covariates, 'weights')

# Print model summary
msm_pp.print_summary()
msm_itt.print_summary()

In [ ]:
import matplotlib.pyplot as plt

def plot_survival_differences(model, treatment_col):
    model.plot_partial_effects(treatment_col, values=[0, 1])
    plt.title('Survival Differences by Treatment')
    plt.show()

# Example usage
plot_survival_differences(msm_pp, 'treatment')
plot_survival_differences(msm_itt, 'treatment')